# **라이브러리 로드**

In [104]:
import pandas as pd  # 데이터 분석 및 처리 라이브러리
import numpy as np  # 수치 연산 및 배열 처리 라이브러리
import tarfile  # 파일 압축 및 압축 해제 모듈
import pickle  # 데이터 직렬화 및 역직렬화 모듈
import os  # 파일 및 디렉토리 경로 관리 모듈

# 데이터 전처리 패키지
from sklearn.decomposition import PCA  # 데이터 차원 축소를 위한 PCA 모듈
from sklearn.manifold import MDS
from sklearn.preprocessing import StandardScaler  # 데이터 정규화를 위한 모듈
from sklearn.preprocessing import LabelEncoder  # 카테고리형 데이터를 수치형으로 변환하는 모듈

# 모델 패키지
from tensorflow.keras.models import Sequential  # 신경망 모델 생성용 API
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D,AvgPool2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.utils import to_categorical  # 클래스 라벨을 one-hot 인코딩
from tensorflow.keras.optimizers import RMSprop  # RMSprop 옵티마이저


# 시각화 패키지
from matplotlib.colors import ListedColormap, BoundaryNorm  # 사용자 정의 색상 맵 및 범위 설정
import matplotlib.patches as mpatches  # 범례 항목 생성
import matplotlib.pyplot as plt  # 데이터 시각화 라이브러리
import seaborn as sns  # 고급 시각화를 위한 라이브러리

# **차원 축소 - 변수 추출법**

## **1. 주성분 분석(PCA)**

### **데이터 불러오기**

breast-cancer.csv
- 유방 종양의 30개 물리적 특성 관련 지표
- 목표 변수 : Diagnosis(악성M/양성B)

In [105]:
# Breast cancer dataset
data = pd.read_csv("../data/breast-cancer.csv")  # 유방암 데이터셋을 CSV 파일에서 읽어옴

In [ ]:
data.head()

In [107]:
# 독립 변수 (특징 데이터)와 종속 변수 (레이블 데이터) 분리
X_canc = data.drop(columns=['id', 'diagnosis', 'Unnamed: 32'], errors='ignore')  
# 'id', 'diagnosis', 'Unnamed: 32' 컬럼 제거 (오류 발생 시 무시)

y_canc = data['diagnosis']  # 진단 결과를 종속 변수로 설정

### **시각화 함수**

In [108]:
def plot_labelled_scatter(X, y, class_labels, s):
    le = LabelEncoder()  # 라벨 데이터를 숫자로 변환하는 인코더 초기화
    y_encoded = le.fit_transform(y)  # 클래스 레이블('M', 'B' 등)을 숫자로 변환
    num_labels = len(class_labels)  # 클래스 레이블의 총 개수

    # 데이터의 x축과 y축 범위 설정
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1

    # 마커와 색상 설정
    marker_array = ['o', '^', '*']  # 각 클래스의 마커 스타일
    color_array = ['#FFFF00', '#00AAFF', '#000000', '#FF00AA']  # 각 클래스의 색상
    cmap_bold = ListedColormap(color_array)  # 강한 색상을 위한 컬러맵 생성
    bnorm = BoundaryNorm(np.arange(0, num_labels + 1, 1), ncolors=num_labels)  # 클래스 경계를 위한 색상 정규화

    plt.figure(figsize=s)  # 그래프 크기 설정

    # 산점도 그리기
    plt.scatter(X[:, 0], X[:, 1], s=80, c=y_encoded, cmap=cmap_bold, norm=bnorm,
                alpha=0.4, edgecolor='black', lw=1)  # 점 크기, 색상, 투명도 및 테두리 설정

    # 그래프의 상단과 오른쪽 축 숨기기
    sp = plt.gca().spines  # 현재 축의 경계를 가져옴
    sp['top'].set_visible(False)  # 상단 경계 숨기기
    sp['right'].set_visible(False)  # 오른쪽 경계 숨기기

    plt.grid(which='both', color='lightslategrey', alpha=0.3)  # 격자 표시 설정

    # x축과 y축 범위 설정
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)

    # 범례 추가
    h = []  # 범례 핸들 리스트
    for c in range(0, num_labels):
        h.append(mpatches.Patch(color=color_array[c], label=class_labels[c]))  # 각 클래스의 색상과 레이블 추가
    plt.legend(handles=h, fontsize=15, frameon=False)  # 범례 스타일 설정

### **1-1. PCA 적용**

In [ ]:
canc_norm = StandardScaler().fit(X_canc).transform(X_canc)  # 데이터를 평균 0, 분산 1로 표준화하여 PCA 적용 전에 정규화

pca = PCA(n_components=2).fit(canc_norm)  # PCA(주성분 분석)를 사용하여 데이터의 주성분 2개를 추출하도록 모델을 학습

canc_pca = pca.transform(canc_norm)  # 학습된 PCA 모델로 데이터를 변환하여 2개의 주성분으로 차원을 축소

print('Number of Features in Breast Cancer DataSet Before PCA : {}\n\nNumber of Features in Breast Cancer DataSet After PCA : {}'
      .format(X_canc.shape[1], canc_pca.shape[1]))  # PCA 전후의 데이터 특징 수 출력 (PCA 적용 전의 특징 수와 축소 후의 특징 수 비교)

### **1-2 PCA 결과 시각화**

In [ ]:
plot_labelled_scatter(canc_pca, y_canc, ['M', 'B'],(15,9)) 

plt.xlabel('First principal component',fontsize=15)
plt.ylabel('Second principal component',fontsize=15)
plt.title('Breast Cancer Dataset PCA (n_components = 2)',fontsize=17)

### **1-3 PCA 결과와 기존 feature의 상관관계 확인**

In [ ]:
fig = plt.figure(figsize=(20, 9))  
# 시각화를 위한 큰 크기의 figure 생성 (20x9)

plt.imshow(pca.components_, interpolation='none', cmap='viridis')  
# PCA 주성분 행렬을 시각화. 각 특징이 주성분에 기여하는 정도를 색상으로 표현

feature_names = list(X_canc.columns)  
# PCA의 입력 데이터에서 사용된 특징 이름을 가져옴

# x축과 y축에 특징 이름과 주성분 이름 설정
plt.gca().set_xticks(np.arange(len(feature_names)))  
plt.gca().set_yticks(np.arange(2))  
plt.gca().set_xticklabels(feature_names, rotation=90, fontsize=14)  
plt.gca().set_yticklabels(['Dimension 1', 'Dimension 2'], fontsize=14)  

# 컬러바 추가 (색상 범위를 나타냄)
plt.colorbar(orientation='horizontal', ticks=[pca.components_.min(), 0, 
                                              pca.components_.max()], pad=0.5)  
# 컬러바를 수평으로 표시하고, 최소값, 0, 최대값으로 눈금 설정

***

## **2. Multi-Dimensional Scaling (MDS)**

In [112]:
mds = MDS(n_components=2).fit(canc_norm)  # PCA(주성분 분석)를 사용하여 데이터의 주성분 2개를 추출하도록 모델을 학습

canc_mds = mds.fit_transform(canc_norm)  # 학습된 PCA 모델로 데이터를 변환하여 2개의 주성분으로 차원을 축소

In [ ]:
plot_labelled_scatter(canc_mds, y_canc, ['M', 'B'],(15,9)) 

plt.xlabel('First principal component',fontsize=15)
plt.ylabel('Second principal component',fontsize=15)
plt.title('Breast Cancer Dataset PCA (n_components = 2)',fontsize=17)

***

## **3. 이미지 데이터에서의 PCA 적용**
#### data : cifar10

### **데이터 로드**

In [ ]:
# CIFAR-10 tar.gz 파일의 경로를 정의
cifar10_tar_path = '../data/cifar-10-python.tar.gz'

# tar.gz 파일 추출
with tarfile.open(cifar10_tar_path, 'r:gz') as tar:
    tar.extractall()  # 파일을 현재 디렉토리로 추출
    print("Files extracted.")  # 파일이 추출되었음을 출력

# 배치 데이터를 파일에서 로드하는 함수 정의
def load_batch(file_path):
    with open(file_path, 'rb') as f:
        data_dict = pickle.load(f, encoding='bytes')  # pickle을 사용하여 파일 로드
        # 이미지 데이터와 레이블 추출
        data = data_dict[b'data']  # 이미지 데이터
        labels = data_dict[b'labels']  # 레이블 데이터
        # 데이터를 이미지 형식(N, H, W, C)으로 변환 및 재배치
        data = data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
        labels = np.array(labels)  # 레이블을 NumPy 배열로 변환
    return data, labels  # 변환된 이미지와 레이블 반환

# CIFAR-10 파일이 추출된 디렉토리
cifar10_dir = 'cifar-10-batches-py'

# 훈련 데이터 로드
x_train = []  # 훈련 이미지 데이터를 저장할 리스트
y_train = []  # 훈련 레이블 데이터를 저장할 리스트
for i in range(1, 6):  # 1번 배치부터 5번 배치까지 반복
    batch_file = os.path.join(cifar10_dir, f'data_batch_{i}')  # 배치 파일 경로
    data, labels = load_batch(batch_file)  # 배치 데이터 로드
    x_train.append(data)  # 이미지 데이터를 리스트에 추가
    y_train.append(labels)  # 레이블 데이터를 리스트에 추가

# 모든 훈련 배치를 연결하여 단일 배열로 변환
x_train = np.concatenate(x_train)  # 훈련 이미지 데이터 연결
y_train = np.concatenate(y_train)  # 훈련 레이블 데이터 연결

# 테스트 데이터 로드
test_batch_file = os.path.join(cifar10_dir, 'test_batch')  # 테스트 배치 파일 경로
x_test, y_test = load_batch(test_batch_file)  # 테스트 데이터 로드

### **데이터 확인**

In [ ]:
#데이터 분리
print('Traning data shape:', x_train.shape)
print('Testing data shape:', x_test.shape)

In [ ]:
y_train.shape,y_test.shape

In [ ]:
classes = np.unique(y_train)
nClasses = len(classes)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

In [134]:
# 라벨 사전 정의
label_dict = {
 0: 'airplane',
 1: 'automobile',
 2: 'bird',
 3: 'cat',
 4: 'deer',
 5: 'dog',
 6: 'frog',
 7: 'horse',
 8: 'ship',
 9: 'truck',
}

In [ ]:
# 데이터 확인을 위한 시각화
plt.figure(figsize=[5,5])  # 그래프 크기 설정

# 훈련 데이터의 첫 번째 이미지 출력
plt.subplot(121)  # 첫 번째 그래프 (1행 2열 중 첫 번째 위치)
curr_img = np.reshape(x_train[0], (32, 32, 3))  # 첫 번째 훈련 데이터를 (32, 32, 3) 형식으로 재구성
plt.imshow(curr_img)  # 이미지를 표시
print(plt.title("(Label: " + str(label_dict[y_train[0]]) + ")"))  # 레이블 출력

# 테스트 데이터의 첫 번째 이미지 출력
plt.subplot(122)  # 두 번째 그래프 (1행 2열 중 두 번째 위치)
curr_img = np.reshape(x_test[0], (32, 32, 3))  # 첫 번째 테스트 데이터를 (32, 32, 3) 형식으로 재구성
plt.imshow(curr_img)  # 이미지를 표시
print(plt.title("(Label: " + str(label_dict[y_test[0]]) + ")"))  # 레이블 출력

### **데이터 Scaling**

In [ ]:
#scaling
x_train = x_train/255.0
x_test = x_test/255
print(np.min(x_train),np.max(x_train))
print(np.min(x_test),np.max(x_test))

In [137]:
#3072개의 픽셀
x_train_flat = x_train.reshape(-1,3072)
x_test_flat = x_test.reshape(-1,3072)

In [ ]:
# 특징 이름 생성 (픽셀 값에 대해 열 이름 설정)
feat_cols_tr = ['pixel'+str(i) for i in range(x_train_flat.shape[1])]  # x_train_flat의 열 수만큼 'pixel0', 'pixel1', ... 형식의 이름 생성
feat_cols_te = ['pixel'+str(i) for i in range(x_test_flat.shape[1])]  # x_test_flat의 열 수만큼 'pixel0', 'pixel1', ... 형식의 이름 생성

# CIFAR-10 데이터를 DataFrame으로 변환
x_train_flat = pd.DataFrame(x_train_flat, columns=feat_cols_tr)  # x_train_flat 데이터를 열 이름과 함께 DataFrame으로 변환
x_train_flat['label'] = y_train  # 레이블 데이터를 DataFrame의 새로운 열로 추가
x_test_flat = pd.DataFrame(x_test_flat, columns=feat_cols_te)  # x_test_flat 데이터를 열 이름과 함께 DataFrame으로 변환
x_test_flat['label'] = y_test  # 레이블 데이터를 DataFrame의 새로운 열로 추가

# DataFrame 크기 출력
print('Size of the dataframe: {}'.format(x_train_flat.shape))  # DataFrame의 크기 (행, 열) 출력
print('Size of the dataframe: {}'.format(x_test_flat.shape))  # DataFrame의 크기 (행, 열) 출력

In [ ]:
x_train_flat.head()

In [ ]:
x_test_flat.head()

### **PCA 적용**

In [141]:
# PCA 모델 초기화
pca_cifar = PCA(n_components=2)  # CIFAR 데이터셋의 차원을 2개 주성분으로 축소하기 위해 PCA 객체 생성

# 학습 데이터에 PCA 적용
pca_train_cifar = pca_cifar.fit_transform(x_train_flat.iloc[:, :-1])  
# 학습 데이터의 마지막 열(레이블)을 제외한 모든 열에 대해 PCA 학습(fit)과 변환(transform) 수행
# 결과는 2차원으로 축소된 학습 데이터

# 테스트 데이터에 PCA 적용
pca_test_cifar = pca_cifar.fit_transform(x_test_flat.iloc[:, :-1])  
# 테스트 데이터의 마지막 열(레이블)을 제외한 모든 열에 대해 동일한 방식으로 PCA 학습 및 변환 수행
# 결과는 2차원으로 축소된 테스트 데이터


In [ ]:
principal_cifar_Df = pd.DataFrame(data = pca_train_cifar, columns = ['principal component 1', 'principal component 2'])
principal_cifar_Df['y'] = y_train

principal_cifar_Df.head()

In [ ]:
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="principal component 1", y="principal component 2",
    hue="y",
    palette=sns.color_palette("hls", 10),
    data=principal_cifar_Df,
    legend="full",
    alpha=0.3
)


### **PCA 효과성 검증**

In [144]:
# 학습 데이터와 테스트 데이터의 마지막 열 제거
x_train_flat_2 = x_train_flat.iloc[:, :-1]  # 학습 데이터에서 마지막 열(레이블)을 제외한 부분만 선택
x_test_flat_2 = x_test_flat.iloc[:, :-1]  # 테스트 데이터에서 마지막 열(레이블)을 제외한 부분만 선택

In [ ]:
pca = PCA(n_components=0.9) # 데이터의 분산을 90% 유지하면서 차원 축소 수행
pca.fit(x_train_flat_2)

In [ ]:
print(pca.n_components_) # PCA에 의해 선택된 주성분의 갯수 확인 
pca_input_size = len(pca.components_)

In [ ]:
# 학습 데이터에 PCA 적용
train_img_pca = pca.transform(x_train_flat_2)  
# PCA를 학습 데이터 전체에 대해 학습(fit) 및 변환(transform)하여 주성분 공간으로 축소

# 테스트 데이터에 PCA 적용
test_img_pca = pca.transform(x_test_flat_2)  
# PCA를 테스트 데이터 전체에 대해 학습 및 변환하여 주성분 공간으로 축소


In [149]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
x_train_flat_2

In [153]:
batch_size = 128
num_classes = 10
epochs = 40

In [ ]:
#원본 학습
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(3072,)))
model.add(Dropout(0.3))  # Dropout 추가
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.3))  # Dropout 추가
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train_flat_2, y_train,batch_size=batch_size,epochs=epochs,verbose=1,
                    validation_data=(x_test_flat_2, y_test))

In [ ]:
#pca 학습
model_pca = Sequential()
model_pca.add(Dense(1024, activation='relu', input_shape=(pca_input_size,)))
model_pca.add(Dropout(0.5))  # Dropout 추가
model_pca.add(Dense(1024, activation='relu'))
model_pca.add(Dropout(0.5))  # Dropout 추가
model_pca.add(Dense(512, activation='relu'))
model_pca.add(Dense(256, activation='relu'))
model_pca.add(Dense(num_classes, activation='softmax'))

model_pca.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history_pca = model_pca.fit(train_img_pca, y_train,batch_size=batch_size,epochs=epochs,verbose=1,
                    validation_data=(test_img_pca, y_test))

In [ ]:
# Plotting the accuracy of the two models
plt.figure(figsize=(14, 6))

# Plot training accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Original Model - Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Original Model - Validation Accuracy')
plt.plot(history_pca.history['accuracy'], label='PCA Model - Training Accuracy')
plt.plot(history_pca.history['val_accuracy'], label='PCA Model - Validation Accuracy')
plt.title('Model Accuracy Comparison')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()